In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [10]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

Use Beautiful Soup to scrape data from wikipedia

Extract data from wikipedia in table

In [14]:
area_table = soup.find('table')
fields = area_table.find_all('td')

Create dataframe of postcode, borough and neighborhood

In [15]:
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())

In [16]:
df_area = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_area.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_area.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove "Not assigned" values

In [20]:
df_area['Borough'].replace('Not assigned', np.nan, inplace=True)
df_area.dropna(subset=['Borough'], inplace=True)

In [24]:
df_area.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Group boroughs by postcode

In [28]:
df_area_code = df_area.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_area_code.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_area_code.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Query for any Borough that has a Neighbourhood value of "Not assigned"

In [39]:
df_area_code.query('Neighbourhood == "Not assigned"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


Update Neighbourhood to match name of Borough

In [42]:
df_area_code.loc[df_area_code.Neighbourhood == 'Not assigned', "Neighbourhood"] = df_area_code.Borough
df_area_code.query('Postcode == "M7A"')

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


Print Dataframe shape

In [45]:
df_area_code.shape

(103, 3)

Save updated data to csv file

In [46]:
df_area_code.to_csv("Toronto_Area_Code", index = False)